NLP From Scratch: Translation with a Sequence to Sequence Network
===============================================================================
##### originally written by: Sean Robertson
##### modified by: Vinothkumar



##### Ref: https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html
##### modified to English to Tamil translation

In [206]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [207]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [208]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[.!?]+", r" ", s) #modifiled for tamil letters
    return s.strip()

In [209]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')


    #Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]


    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [210]:
MAX_LENGTH = 20

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

# def filterPair(p):
#     return len(p[0].split(' ')) < MAX_LENGTH and \
#         len(p[1].split(' ')) < MAX_LENGTH and \
#         p[1].startswith(eng_prefixes)
    
def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [211]:
def prepareData(lang1, lang2, reverse=True):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    
    #commented (filter pair did not work for tamil)
    pairs = filterPairs(pairs)
    
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('eng', 'tamil', False)
print(random.choice(pairs))

Reading lines...
Read 392 sentence pairs
Trimmed to 389 sentence pairs
Counting words...
Counted words:
eng 645
tamil 920
['she sat next to me', 'அவள எனககு அருகில அமரநதாள']


In [212]:
pairs[0:10]

[['i love you', 'நான உனனை காதலிககிறேன'],
 ['where are you', 'ந எஙகே இருககிறாய'],
 ['where are you going', 'நஙகள எஙகே போகிறரகள'],
 ['i have to go to the toilet', 'நான கழிவறைககுச செலல வேணடும'],
 ['"i like the movie ""me""  "', 'நான எனறு படம விருமபுகிறேன'],
 ['something extraordinary happened to me this morning',
  'ஒரு விசிததிரமான ஒனறு இனறு காலை எனககு நடநதது'],
 ['would you like to improve your english',
  'உஙகள ஆஙகிலம மேமபடுதத வேணடுமென விருமபுகிறரகளா'],
 ['i tried to understand what had happened',
  'எனன நடநதது எனபதை தெரிநது கொளள விருமபினேன'],
 ['tom tried not to cry', 'டோம அழாமல இருகக முயனறான'],
 ["where's tom been all day", 'நாள முழுதும டோம எஙகிருநதான']]

In [213]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_p=0.1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input):
        embedded = self.dropout(self.embedding(input))
        output, hidden = self.gru(embedded)
        return output, hidden

In [214]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden  = self.forward_step(decoder_input, decoder_hidden)
            decoder_outputs.append(decoder_output)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        return decoder_outputs, decoder_hidden, None # We return `None` for consistency in the training loop

    def forward_step(self, input, hidden):
        output = self.embedding(input)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.out(output)
        return output, hidden

In [215]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(1, -1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

def get_dataloader(batch_size):
    input_lang, output_lang, pairs = prepareData('eng', 'tamil', False)

    n = len(pairs)
    input_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)
    target_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)

    for idx, (inp, tgt) in enumerate(pairs):
        inp_ids = indexesFromSentence(input_lang, inp)
        tgt_ids = indexesFromSentence(output_lang, tgt)
        inp_ids.append(EOS_token)
        tgt_ids.append(EOS_token)
        input_ids[idx, :len(inp_ids)] = inp_ids
        target_ids[idx, :len(tgt_ids)] = tgt_ids

    train_data = TensorDataset(torch.LongTensor(input_ids).to(device),
                               torch.LongTensor(target_ids).to(device))

    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
    return input_lang, output_lang, train_dataloader

In [216]:
def train_epoch(dataloader, encoder, decoder, encoder_optimizer,
          decoder_optimizer, criterion):

    total_loss = 0
    for data in dataloader:
        input_tensor, target_tensor = data

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden, target_tensor)

        loss = criterion(
            decoder_outputs.view(-1, decoder_outputs.size(-1)),
            target_tensor.view(-1)
        )
        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)

In [217]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [218]:
def train(train_dataloader, encoder, decoder, n_epochs, learning_rate=0.001,
               print_every=100, plot_every=100):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    for epoch in range(1, n_epochs + 1):
        loss = train_epoch(train_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if epoch % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, epoch / n_epochs),
                                        epoch, epoch / n_epochs * 100, print_loss_avg))

        if epoch % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [219]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [228]:
def evaluate(encoder, decoder, sentence, input_lang, output_lang):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, decoder_hidden, _ = decoder(encoder_outputs, encoder_hidden)

        _, topi = decoder_outputs.topk(1)
        decoded_ids = topi.squeeze()

        decoded_words = []
        for idx in decoded_ids:
            if idx.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            decoded_words.append(output_lang.index2word[idx.item()])
    return decoded_words

In [231]:
def evaluateRandomly(encoder, decoder, n=100):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, pair[0], input_lang, output_lang)
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [222]:
hidden_size = 128
batch_size = 16

input_lang, output_lang, train_dataloader = get_dataloader(batch_size)

encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder = DecoderRNN(hidden_size, output_lang.n_words).to(device)

train(train_dataloader, encoder, decoder, 100, print_every=5, plot_every=5)

Reading lines...
Read 392 sentence pairs
Trimmed to 389 sentence pairs
Counting words...
Counted words:
eng 645
tamil 920
0m 1s (- 0m 26s) (5 5%) 2.3450
0m 2s (- 0m 24s) (10 10%) 1.3701
0m 4s (- 0m 23s) (15 15%) 1.2589
0m 5s (- 0m 21s) (20 20%) 1.1443
0m 6s (- 0m 19s) (25 25%) 1.0151
0m 7s (- 0m 18s) (30 30%) 0.8749
0m 9s (- 0m 17s) (35 35%) 0.7412
0m 10s (- 0m 15s) (40 40%) 0.6060
0m 11s (- 0m 14s) (45 45%) 0.4875
0m 13s (- 0m 13s) (50 50%) 0.3925
0m 14s (- 0m 11s) (55 55%) 0.3155
0m 15s (- 0m 10s) (60 60%) 0.2571
0m 16s (- 0m 9s) (65 65%) 0.2177
0m 18s (- 0m 7s) (70 70%) 0.1877
0m 19s (- 0m 6s) (75 75%) 0.1630
0m 20s (- 0m 5s) (80 80%) 0.1422
0m 22s (- 0m 3s) (85 85%) 0.1245
0m 23s (- 0m 2s) (90 90%) 0.1094
0m 24s (- 0m 1s) (95 95%) 0.0966
0m 25s (- 0m 0s) (100 100%) 0.0850


Set dropout layers to `eval` mode


In [232]:
encoder.eval()
decoder.eval()
evaluateRandomly(encoder, decoder)

> i like orange
= எனககு ஆரஞசு பிடிககும
< எனககு சிவபபு <EOS>

> raise your hand
= கையைத தூககு
< கையைத <EOS>

> i'm getting old
= வயசு ஆகது
< வயசு <EOS>

> i'll walk
= நான நடபபேன
< நான தூஙகுகிறேன <EOS>

> rain is like ambrosia, as the world cannot exist without rain
= வானின றுலகம வழஙகி வருதலால தானமிழதம எனறுணரற பாறறு
< பூசனை வானோரககும ஈணடு <EOS>

> when he spoke, everyone became silent
= அவன பேசியப பொழுது எலலோரும அமைதி காததாரகள
< வேணடுதல வானோரககும ஈணடு <EOS>

> it needs help right now
= அதறகு இபபோது உதவி தேவை
< சிறபபொடு ஒழுகக நெறிநினறார நடுவாழ வார <EOS>

> do you have a lot of pens
= உனனிடம நிறைய பேனாககள இருககினறனவா
< நஙகள கணிசமான சமபளம தர வேணடும <EOS>

> would you like to improve your english
= உஙகள ஆஙகிலம மேமபடுதத வேணடுமென விருமபுகிறரகளா
< சிறபபொடு பூசனை செலலாது வானோரககும ஈணடு <EOS>

> i'll leave that to you
= நான அதை உனனிடம விடடு விடுகிறேன
< நான தூஙகவிலலை  " <EOS>

> eat slowly
= மெதுவாக சாபபிடுஙகள
< ந நினைககிறாய <EOS>

> tom has been unconscious for three days
= டோம மூனறு நாட